In [1]:
from model import *
from data_preprocessing import *

In [2]:
labels = ['ABSZ', 'CPSZ', 'FNSZ', 'GNSZ', 'SPSZ', 'TCSZ', 'TNSZ']
labels_2_num = {labels[i]: i for i in range(len(labels))}

use_gpu = False

In [3]:
preprocess_1()

NameError: name 'tqdm_notebook' is not defined